In [28]:
import sys
sys.path.append("../")


import os
%matplotlib inline
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from patsy import dmatrices
from openpyxl import load_workbook

from linearmodels import PanelOLS

from library import print_statistics

In [29]:
def coef_with_stars(coef, pvalue):
    coef = round(coef, 2)
    if pvalue >.05:
        coef = str(coef)
    if pvalue <= .05:
        coef = str(coef) + '*'
    if pvalue <= .01:
        coef = coef + '*'
    if pvalue <= .001:
        coef = coef + '*'
    return(coef)
test = coef_with_stars(9.1568, 0.8523)
test

'9.16'

In [30]:
def bonferroni(n_tests, coef, pvalue):
    coef = round(coef, 2)
    if pvalue >(.05/n_tests):
        coef = str(coef)
    if pvalue <= (.05/n_tests):
        coef = str(coef) + '*'
    if pvalue <= (.01/n_tests):
        coef = coef + '*'
    if pvalue <= (.001/n_tests):
        coef = coef + '*'
    return(coef)
test = bonferroni(4, .1, .005)
test

'0.1*'

In [31]:
def format_se(se):
    if se < .005:
        se = '(0.00)'
    else:
        se = '(' + str(round(se, 2)) + ')'
    return se
test = format_se(.0053)
test

'(0.01)'

In [32]:
data_path = '/Users/kylieleblancKylie/domino/dofis/data/'
table_path = '/Users/kylieleblancKylie/domino/dofis/results/Who Needs Rules/'
data = pd.read_csv(os.path.join(data_path, 'clean', 'gdid_subject.csv'),
                  sep=",", low_memory= False)
#load(data)
print(data[data.doi == True].district.nunique())
data.sample()

808


,Unnamed: 0,campus,year,test,score,score_std,campname,campischarter,district,distname,...,pre5,pre4,pre3,pre2,pre1,pre0,post1,post2,post3,test_by_year
102185,128578,234902102,2012,m_5th_avescore,1625.0,0.426881,CANTON INT,N,234902,CANTON ISD,...,1,0,0,0,0,0,0,0,0,m_5th_avescore2012


In [33]:
data.doi_year.value_counts()

2018.0    133956
2017.0     89082
2019.0     30453
2020.0      2906
Name: doi_year, dtype: int64

In [34]:
# phase-in effect (yearpost)
data[['year', 'doi_year', 'treatpost', 'yearpost']].sample(10)

,year,doi_year,treatpost,yearpost
198794,2017,2018.0,False,0.0
49120,2016,2018.0,False,0.0
128383,2013,2018.0,False,0.0
221959,2018,2017.0,True,1.0
134618,2013,2018.0,False,0.0
51373,2016,2018.0,False,0.0
23471,2014,2018.0,False,0.0
202497,2017,2017.0,True,0.0
80430,2018,2018.0,True,0.0
241800,2019,2018.0,True,1.0


In [35]:
data.yearpost.value_counts()

0.0    216448
1.0     28412
2.0     11537
Name: yearpost, dtype: int64

## Pretrends - yearpre

In [36]:
# Pre-trends
data[['year', 'doi_year', 'treatpost', 'yearpost', 'yearpre']].sample(5)

,year,doi_year,treatpost,yearpost,yearpre
227741,2019,2017.0,True,2.0,0.0
44347,2015,2018.0,False,0.0,-3.0
156592,2015,2018.0,False,0.0,-3.0
186154,2016,2018.0,False,0.0,-2.0
149407,2014,2017.0,False,0.0,-3.0


In [37]:
data.yearpre.value_counts()

 0.0    72073
-1.0    32297
-2.0    32092
-3.0    31836
-4.0    31640
-5.0    31408
-6.0    20621
-7.0     4080
-8.0      350
Name: yearpre, dtype: int64

In [38]:
#convert year to datetime
df = data.reset_index()
df['year'] = pd.to_datetime(df['year'], format='%Y')
#add column year to index
df = data.set_index(['year', 'campus'])
#swap indexes
df.index = df.index.swaplevel(0,1)
df[['district', 'doi_year','treatpost']].sample(5, random_state = 8)

,,district,doi_year,treatpost
campus,year,,,
226903102,2013,226903,2018.0,False
220908202,2014,220908,2017.0,False
61903102,2013,61903,2017.0,False
20907106,2017,20907,2018.0,False
220901121,2013,220901,2017.0,False


# Specifications

In [39]:
# Get table ready
file = table_path + 'table3_gdid_and_event.xlsx'
wb = load_workbook(file)
ws = wb.active

## Simple GDID

In [40]:
test = pd.Categorical(df.test)
mod = PanelOLS.from_formula('score_std ~ + 1 + treatpost + students_hisp + students_num + C(test_by_year) + EntityEffects', df)
res = mod.fit(cov_type='clustered', cluster_entity=True)
print(res)
ws.cell(row= 3, column= 2).value = coef_with_stars(res.params['treatpost[T.True]'], res.pvalues['treatpost[T.True]'])
ws.cell(row= 4, column= 2).value = format_se(res.std_errors['treatpost[T.True]'])

                          PanelOLS Estimation Summary                           
Dep. Variable:              score_std   R-squared:                        0.2155
Estimator:                   PanelOLS   R-squared (Between):             -0.2055
No. Observations:              256397   R-squared (Within):               0.2155
Date:                Mon, Dec 30 2019   R-squared (Overall):              0.0375
Time:                        21:24:52   Log-likelihood                -1.933e+05
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      563.39
Entities:                        6033   P-value                           0.0000
Avg Obs:                       42.499   Distribution:              F(122,250242)
Min Obs:                       1.0000                                           
Max Obs:                       120.00   F-statistic (robust):             299.92
                            

## GDID with Trends

In [42]:
mod = PanelOLS.from_formula('score_std ~ + 1 + treatpost + yearpost + yearpre + students_hisp + students_num + C(test_by_year) + EntityEffects', df)
#mod = PanelOLS.from_formula('score_std ~ + 1 + treatpost + yearpost + yearpre + students_hisp + students_num + TimeEffects + EntityEffects', df)
res = mod.fit(cov_type='clustered', cluster_entity=True)
print(res)
#intercept = res.params['Intercept']
jump = res.params['treatpost[T.True]']
preslope = res.params['yearpre']
preslope_se = res.std_errors['yearpre']
postslope = res.params['yearpost']
post_slope = res.std_errors['yearpost']
ws.cell(row= 6, column= 2).value = coef_with_stars(res.params['treatpost[T.True]'], res.pvalues['treatpost[T.True]'])
ws.cell(row= 7, column= 2).value = format_se(res.std_errors['treatpost[T.True]'])
ws.cell(row= 8, column= 2).value = coef_with_stars(res.params['yearpost'], res.pvalues['yearpost'])
ws.cell(row= 9, column= 2).value = format_se(res.std_errors['yearpost'])
ws.cell(row= 10, column= 2).value = coef_with_stars(res.params['yearpre'], res.pvalues['yearpre'])
ws.cell(row= 11, column= 2).value = format_se(res.std_errors['yearpre'])
wb.save(file)

AbsorbingEffectError: 
The model cannot be estimated. The included effects have fully absorbed
one or more of the variables. This occurs when one or more of the dependent
variable is perfectly explained using the effects included in the model.

The following variables or variable combinations have been fully absorbed
or have become perfectly collinear after effects are removed:

          Intercept, C(test_by_year)[T.alg_avescore2013], C(test_by_year)[T.alg_avescore2014], C(test_by_year)[T.alg_avescore2015], C(test_by_year)[T.alg_avescore2016], C(test_by_year)[T.alg_avescore2017], C(test_by_year)[T.alg_avescore2018], C(test_by_year)[T.alg_avescore2019], C(test_by_year)[T.bio_avescore2013], C(test_by_year)[T.bio_avescore2014], C(test_by_year)[T.bio_avescore2015], C(test_by_year)[T.bio_avescore2016], C(test_by_year)[T.bio_avescore2017], C(test_by_year)[T.bio_avescore2018], C(test_by_year)[T.bio_avescore2019], C(test_by_year)[T.eng1_avescore2013], C(test_by_year)[T.eng1_avescore2014], C(t

# Non-parametric event study

In [ ]:
df[['test_by_year', 'doi_year', 'treatpost', 'yearpost', 'yearpre',
   'pre6', 'pre5', 'pre4', 'pre3', 'pre2', 'pre1', 'pre0',
    'post1', 'post2', 'post3']].groupby(['test_by_year']).agg('mean')

In [ ]:
df[(df.test_by_year == 'alg_avescore2012') & (df.pre0 == 1)][['doi_year', 'treatpost', 'yearpost', 'yearpre',
    'pre5', 'pre4', 'pre3', 'pre2', 'pre1', 'pre0',
    'post1', 'post2', 'post3']]

In [ ]:
df.head()

In [ ]:
mod = PanelOLS.from_formula('score_std ~ + 1 + pre5 + pre4 + pre3 + pre2 + pre1 + post1 + post2 + post3 + students_hisp + students_num + C(test_by_year) + EntityEffects', df)
res = mod.fit(cov_type='clustered', cluster_entity=True)
print(res)
nonparametric = []
nonparametric_se = []
for coef in ['pre5', 'pre4', 'pre3', 'pre2', 'pre1', 'pre0', 'post1', 'post2', 'post3']:
    nonpar = 0
    nonpar_se = 0
    if coef != 'pre0':
        nonpar = res.params[coef]
        nonpar_se = res.std_errors[coef]
    nonparametric.append(nonpar)
    nonparametric_se.append(nonpar_se)
print(nonparametric)
row = 3
for coef in ['post3', 'post2', 'post1', 'pre1', 'pre2', 'pre3', 'pre4', 'pre5']:
    ws.cell(row= row, column= 4).value = coef_with_stars(res.params[coef], res.pvalues[coef])
    row = row + 1
    ws.cell(row= row, column= 4).value = format_se(res.std_errors[coef])  
    row = row + 1
wb.save(file)

In [ ]:
df.pre1.value_counts()

In [ ]:
coef_df = pd.DataFrame({'coef': nonparametric,
                        'err': nonparametric_se,
                        'year': [-5, -4, -3, -2, -1,0 , 1, 2, 3]
                       })
coef_df['lb'] = coef_df.coef - (1.96*coef_df.err)
coef_df['ub'] = coef_df.coef + (1.96*coef_df.err)
coef_df

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
coef_df.plot(x='year', y='coef', kind='bar', 
             ax=ax, color='none', 
             yerr='err', legend=False)
ax.set_ylabel('')
ax.set_xlabel('')
ax.scatter(x=pd.np.arange(coef_df.shape[0]), 
           marker='s', s=120, 
           y=coef_df['coef'], color='black')
ax.axhline(y=0, linestyle='--', color='black', linewidth=4)
ax.xaxis.set_ticks_position('none')
_ = ax.set_xticklabels(['Pre5', 'Pre4', 'Pre3', 'Pre2', 'Pre1', 'Pre0', 'Post1', 'Post2', 'Post3'], 
                       rotation=0)
#ax.set_title('Impact on Student Achievement - Event Study Coefficients', fontsize = 16)

# Create graph (look up how to use predicted values)


In [ ]:
years = [-5, -4, -3, -2, -1, 0, 1, 2, 3]
parametric = []
parametric_lb = []
for year in years:
    par = 0
    if year < 0 :
        par = (year * preslope)
    if year > 0 :
        par = jump + (year * postslope)
    parametric.append(par)
parametric

In [ ]:
nonparametric

In [ ]:
plt.plot(years, parametric)
plt.plot(years, nonparametric)

# Table by Subject

In [ ]:
subjects = ['m_3rd_avescore', 'r_3rd_avescore',
            'm_4th_avescore', 'r_4th_avescore', 
            'm_5th_avescore', 'r_5th_avescore', 
            'm_6th_avescore', 'r_6th_avescore',
            'm_7th_avescore', 'r_7th_avescore',
            'm_8th_avescore', 'r_8th_avescore',
            'alg_avescore', 'bio_avescore', 'eng1_avescore']

In [ ]:
# All Subject Table
file = table_path + 'tableA_effect_by_subject.xlsx'
wb = load_workbook(file)
ws = wb.active

col = 3
for subject in subjects:
    
    df_sub = df[df.test == subject]
    test = pd.Categorical(df_sub.test)
    
    # GDID
    mod = PanelOLS.from_formula('score_std ~ + 1 + treatpost + students_hisp + students_num + C(test_by_year) + EntityEffects', df_sub)
    res = mod.fit(cov_type='clustered', cluster_entity=True)
    ws.cell(row= 4, column= col).value = bonferroni(len(subjects), res.params['treatpost[T.True]'], res.pvalues['treatpost[T.True]'])
    ws.cell(row= 5, column= col).value = format_se(res.std_errors['treatpost[T.True]'])

    # GDID with Trend
    mod = PanelOLS.from_formula('score_std ~ + 1 + treatpost + yearpost + yearpre + students_hisp + students_num + C(test_by_year) + EntityEffects', df_sub)
    #mod = PanelOLS.from_formula('score_std ~ + 1 + treatpost + yearpost + yearpre + students_hisp + students_num + TimeEffects + EntityEffects', df_sub)
    res = mod.fit(cov_type='clustered', cluster_entity=True)
    jump = res.params['treatpost[T.True]']
    preslope = res.params['yearpre']
    preslope_se = res.std_errors['yearpre']
    postslope = res.params['yearpost']
    post_slope = res.std_errors['yearpost']
    ws.cell(row= 7, column= col).value = bonferroni(len(subjects), res.params['treatpost[T.True]'], res.pvalues['treatpost[T.True]'])
    ws.cell(row= 8, column= col).value = format_se(res.std_errors['treatpost[T.True]'])
    ws.cell(row= 9, column= col).value = bonferroni(len(subjects), res.params['yearpost'], res.pvalues['yearpost'])
    ws.cell(row= 10, column= col).value = format_se(res.std_errors['yearpost'])
    ws.cell(row= 11, column= col).value = bonferroni(len(subjects), res.params['yearpre'], res.pvalues['yearpre'])
    ws.cell(row= 12, column= col).value = format_se(res.std_errors['yearpre'])
    wb.save(file)

    # Event Study
    mod = PanelOLS.from_formula('score_std ~ + 1 + pre5 + pre4 + pre3 + pre2 + pre1 + post1 + post2 + post3 + students_hisp + students_num + C(test_by_year) + EntityEffects', df_sub)
    res = mod.fit(cov_type='clustered', cluster_entity=True)
    nonparametric = []
    nonparametric_se = []
    for coef in ['pre5', 'pre4', 'pre3', 'pre2', 'pre1', 'pre0', 'post1', 'post2', 'post3']:
        nonpar = 0
        nonpar_se = 0
        if coef != 'pre0':
            nonpar = res.params[coef]
            nonpar_se = res.std_errors[coef]
        nonparametric.append(nonpar)
        nonparametric_se.append(nonpar_se)
    print(nonparametric)
    row = 14
    for coef in ['post3', 'post2', 'post1', 'pre1', 'pre2', 'pre3', 'pre4', 'pre5']:
        ws.cell(row= row, column= col).value = bonferroni(len(subjects), res.params[coef], res.pvalues[coef])
        row = row + 1
        ws.cell(row= row, column= col).value = format_se(res.std_errors[coef])  
        row = row + 1
    wb.save(file)
    col = col + 1

# Effects without 6th Grade Math

In [ ]:
file = table_path + 'tableA2_gdid_and_event.xlsx'
wb = load_workbook(file)
ws = wb.active


df_limited = df[df.test != 'm_6th_avescore']

test = pd.Categorical(df_limited.test)
mod = PanelOLS.from_formula('score_std ~ + 1 + treatpost + students_hisp + students_num + C(test_by_year) + EntityEffects', df_limited)
res = mod.fit(cov_type='clustered', cluster_entity=True)
print(res)
ws.cell(row= 3, column= 2).value = coef_with_stars(res.params['treatpost[T.True]'], res.pvalues['treatpost[T.True]'])
ws.cell(row= 4, column= 2).value = format_se(res.std_errors['treatpost[T.True]'])

mod = PanelOLS.from_formula('score_std ~ + 1 + treatpost + yearpost + yearpre + students_hisp + students_num + C(test_by_year) + EntityEffects', df_limited)
#mod = PanelOLS.from_formula('score_std ~ + 1 + treatpost + yearpost + yearpre + students_hisp + students_num + TimeEffects + EntityEffects', df_limited)
res = mod.fit(cov_type='clustered', cluster_entity=True)
print(res)
#intercept = res.params['Intercept']
jump = res.params['treatpost[T.True]']
preslope = res.params['yearpre']
preslope_se = res.std_errors['yearpre']
postslope = res.params['yearpost']
post_slope = res.std_errors['yearpost']
ws.cell(row= 6, column= 2).value = coef_with_stars(res.params['treatpost[T.True]'], res.pvalues['treatpost[T.True]'])
ws.cell(row= 7, column= 2).value = format_se(res.std_errors['treatpost[T.True]'])
ws.cell(row= 8, column= 2).value = coef_with_stars(res.params['yearpost'], res.pvalues['yearpost'])
ws.cell(row= 9, column= 2).value = format_se(res.std_errors['yearpost'])
ws.cell(row= 10, column= 2).value = coef_with_stars(res.params['yearpre'], res.pvalues['yearpre'])
ws.cell(row= 11, column= 2).value = format_se(res.std_errors['yearpre'])
wb.save(file)

mod = PanelOLS.from_formula('score_std ~ + 1 + pre5 + pre4 + pre3 + pre2 + pre1 + post1 + post2 + post3 + students_hisp + students_num + C(test_by_year) + EntityEffects', df_limited)
res = mod.fit(cov_type='clustered', cluster_entity=True)
print(res)
nonparametric = []
nonparametric_se = []
for coef in ['pre5', 'pre4', 'pre3', 'pre2', 'pre1', 'pre0', 'post1', 'post2', 'post3']:
    nonpar = 0
    nonpar_se = 0
    if coef != 'pre0':
        nonpar = res.params[coef]
        nonpar_se = res.std_errors[coef]
    nonparametric.append(nonpar)
    nonparametric_se.append(nonpar_se)
print(nonparametric)
row = 3
for coef in ['post3', 'post2', 'post1', 'pre1', 'pre2', 'pre3', 'pre4', 'pre5']:
    ws.cell(row= row, column= 4).value = coef_with_stars(res.params[coef], res.pvalues[coef])
    row = row + 1
    ws.cell(row= row, column= 4).value = format_se(res.std_errors[coef])  
    row = row + 1
wb.save(file)